In [6]:
#Importamos las bibliotecas a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

In [7]:
#Leemos los archivos a trabajar
df1 = pd.DataFrame( pd.read_csv("meteo-nogal-09.csv", engine="python" ) )
df2 = pd.DataFrame( pd.read_csv("soil-nogal-09.csv"))

In [8]:
df1.head()

,DATE,TIME,Unnamed: 2,Unnamed: 3,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,TS,NaN,NaN,NaN,m/s,m/s,m/s,degC,mV,w/m2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-01-09,00:10,1.0,NaN,1.215,0.506,-0.044,10.95,1.159,-176.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-01-09,00:20,1.0,NaN,0.892,0.588,-0.072,10.81,1.159,-176.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-01-09,00:30,1.0,NaN,1.227,0.6850001,-0.06900001,10.34,1.224,-175.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-01-09,00:40,1.0,NaN,1.135,0.7410001,-0.093,9.8,1.258,-182.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df1.dtypes

DATE            object
TIME            object
Unnamed: 2     float64
Unnamed: 3     float64
u_Avg           object
v_Avg           object
w_Avg           object
t_Avg           object
kh20_Avg        object
net_rad_Avg     object
shf1_Avg        object
shf2_Avg        object
vv_Avg          object
airT_Avg        object
rh_Avg         float64
e_sat_Avg       object
e_Avg           object
h2o_hmp_Avg     object
Unnamed: 18    float64
Unnamed: 19     object
Unnamed: 20     object
Unnamed: 21    float64
Unnamed: 22     object
Unnamed: 23     object
Unnamed: 24     object
Unnamed: 25     object
Unnamed: 26     object
Unnamed: 27     object
Unnamed: 28     object
Unnamed: 29     object
Unnamed: 30     object
Unnamed: 31     object
Unnamed: 32     object
Unnamed: 33     object
Unnamed: 34     object
Unnamed: 35     object
dtype: object

In [10]:
df2.dtypes


id                        int64
2 Year_RTM  L             int64
3 Day_RTM  L              int64
4 Hour_Minute_RTM  L      int64
5 T108v_1_AVG  L        float64
6 T108v_2_AVG  L        float64
7 T108v_3_AVG  L        float64
8 T108v_4_AVG  L        float64
9 T108v_5_AVG  L        float64
10 T108v_6_AVG  L       float64
11 T108v_7_AVG  L       float64
12 T108v_8_AVG  L       float64
Tsuelo_10cm             float64
Tsuelo_20cm             float64
Tsuelo_30cm             float64
Tsuelo_40cm             float64
Tsuelo_55cm             float64
Tsuelo_70cm             float64
Tsuelo_85cm             float64
Tsuelo_100cm            float64
21 Tref_AVG  L          float64
22 period_1  L          float64
23 period_2  L          float64
24 period_3  L          float64
25 period_4  L          float64
26 period_5  L          float64
27 period_6  L          float64
28 period_7  L          float64
29 period_8  L          float64
30 TVWC_1  L            float64
                         ...   
35 TVWC_

In [11]:
#Es necesario adaptar los DataFrames para poder trabajar con ellos.
#Eliminamos la fila que dicta las unidades de cada columna en el df1
df1 = df1.drop(df1.columns[df1.columns.str.contains('unnamed:',case=False)],axis=1)
df1 = df1.drop(df1.index[0]).reset_index(drop=True)

In [12]:
#Vemos que el df2 tiene sus unidades de tiempo cada media hora. Necesitamos que los tiempos del df1 sean de la 
#misma manera.
#Creamos una variable DATETIME que junte las columnas DATE y TIME para df1
df1["DATETIME"] = df1["DATE"] +" "+ df1["TIME"]
#Eliminamos las columnas DATE y TIME
df1.drop( ["DATE","TIME"], axis=1, inplace=True )

In [13]:
df1.tail()

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,DATETIME
52633,1.146,-0.171,7.161,1558,24.1,NAN,NAN,1.618,7.829,0.328,1.060,0.348,2.681,NaN,03/01/2010 08:10:00
52634,1.454,-0.057,9.64,1549,64.78,NAN,NAN,1.679,10.43,0.292,1.263,0.369,2.816,NaN,03/01/2010 08:30:00
52635,1.321,-0.141,10.38,1563,76.34,NAN,NAN,1.662,11.47,0.273,1.354,0.369,2.808,NaN,03/01/2010 08:40:00
52636,1.295,-0.094,11.25,1564,97.7,NAN,NAN,1.62,12.59,0.256,1.457,0.373,2.83,NaN,03/01/2010 08:50:00
52637,0.75,-0.025,12.03,1564,120.1,871,NAN,1.054,13.58,0.243,1.554,0.377,2.85,NaN,03/01/2010 09:00:00


In [14]:
#Convertimos nuestra variable DATETIME en una variable FECHA del tipo date
df1['FECHA'] = pd.to_datetime(df1.apply(lambda x: x['DATETIME'], 1), dayfirst=True)
df1 = df1.drop(['DATETIME'], 1)
df1.head(3)

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA
0,1.215,0.506,-0.044,10.95,1.159,-176.5,23.58,-26.72,1.327,9.4,0.899,1.178,1.059,8.12,2009-01-01 00:10:00
1,0.892,0.588,-0.072,10.81,1.159,-176.6,23.73,-26.76,1.079,9.24,0.896,1.166,1.045,8.02,2009-01-01 00:20:00
2,1.227,0.6850001,-0.06900001,10.34,1.224,-175.5,23.61,-26.46,1.421,9.059999,0.896,1.152,1.032,7.926001,2009-01-01 00:30:00


In [15]:
#De nuestra variable FECHA, obtenemos una nueva variable que contabilice los minutos transcurridos
df1['MIN'] = df1['FECHA'].dt.minute
df1.head(6)

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA,MIN
0,1.215,0.506,-0.044,10.95,1.159,-176.5,23.58,-26.72,1.327,9.4,0.899,1.178,1.059,8.12,2009-01-01 00:10:00,10.0
1,0.892,0.588,-0.072,10.81,1.159,-176.6,23.73,-26.76,1.079,9.24,0.896,1.166,1.045,8.02,2009-01-01 00:20:00,20.0
2,1.227,0.6850001,-0.06900001,10.34,1.224,-175.5,23.61,-26.46,1.421,9.059999,0.896,1.152,1.032,7.926001,2009-01-01 00:30:00,30.0
3,1.135,0.7410001,-0.093,9.8,1.258,-182.7,23.41,-26.46,1.384,9.059999,0.874,1.152,1.008,7.737,2009-01-01 00:40:00,40.0
4,0.7550001,1.064,-0.149,8.87,1.332,-184.8,23.22,-26.39,1.359,8.63,0.859,1.119,0.961,7.39,2009-01-01 00:50:00,50.0
5,0.8850001,0.992,-0.167,8.76,1.45,-183.4,23.33,-26.98,1.39,8.34,0.840,1.097,0.9210001,7.089,2009-01-01 01:00:00,0.0


In [16]:
#Filtramos ahora las filas de nuestro df1 para los valores de cada media hora, es decir, aquellos donde la variable
#'MIN' es igual a 30.0 (media hora) o a 0.0 (una hora); filtramos también todos aquellos que no sean del año 2009.
df1 = df1[((df1["MIN"] == 30.0) | (df1["MIN"] == 0.0)) & (df1["FECHA"] < '2010-01-01 00:30:00')]
df1 = df1.reset_index(drop=True)
df1.tail()

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA,MIN
17441,1.676,-0.139,14.36,1113,-79.21,NAN,NAN,3.067,14.7,0.407,1.672,0.681,5.126,NaN,2009-12-31 22:00:00,0.0
17442,1.196,-0.12,14.17,1170,-78.89,NAN,NAN,2.683,14.63,0.388,1.664,0.645,4.855,NaN,2009-12-31 22:30:00,30.0
17443,0.454,-0.074,12.63,1060,-72.47,NAN,NAN,2.419,13,0.472,1.497,0.707,5.354,NaN,2009-12-31 23:00:00,0.0
17444,0.746,-0.116,11.9,1043,-74.88,NAN,NAN,2.38,12.3,0.5,1.430,0.715,5.428,NaN,2009-12-31 23:30:00,30.0
17445,0.72,-0.078,11.02,1071,-70.44,NAN,NAN,2.068,11.28,0.521,1.336,0.696,5.299,NaN,2010-01-01 00:00:00,0.0


In [17]:
#Para nuestro df2, necesitamos acomodar los minutos de igual manera. Para esto, necesitamos separar las horas de manera
#que el tiempo se denote "HORA:MINUTOS".
#Convertimos la variable '4 Hour_Minute_RTM  L' y '2 Year_RTM  L ' a object
df2['4 Hour_Minute_RTM  L'] = df2['4 Hour_Minute_RTM  L'].astype(str)
df2['2 Year_RTM  L'] = df2['2 Year_RTM  L'].astype(str)

In [18]:

#Llenamos un arreglo de horas y minutos para cada dato

h=[]   #Arreglo de horas.
m=[]   #Arreglo de minutos.

for i in range (0, len(df2)):
    if (len(str(df2['4 Hour_Minute_RTM  L'][i]))==4):
        if (str(df2['4 Hour_Minute_RTM  L'][i])[0:2]=='24'):
            h.append('00')
            m.append('00')
        else:
            h.append(str(df2['4 Hour_Minute_RTM  L'][i])[0:2])
            m.append(str(df2['4 Hour_Minute_RTM  L'][i])[2:4])
    elif (len(str(df2['4 Hour_Minute_RTM  L'][i]))==3):
            h.append(str(df2['4 Hour_Minute_RTM  L'][i])[0:1])
            m.append(str(df2['4 Hour_Minute_RTM  L'][i])[1:3])
    elif (len(str(df2['4 Hour_Minute_RTM  L'][i]))==2):
            h.append('00')
            m.append(str(df2['4 Hour_Minute_RTM  L'][i])[0:2])

In [19]:
#Creamos una variable del tipo string para la fecha que incluya el año, el número de día, la hora y el minuto
t = []   #Arreglo de fechas
for i in range (0,len(df2)):
    t.append( h[i]+':'+m[i])

In [20]:
df2['TIEMPO'] = t
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,THETA_30cm.1,THETA_40cm.1,THETA_55cm.1,THETA_70cm.1,THETA_85cm.1,THETA_100cm.1,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO
17477,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00
17478,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:30
17479,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:00
17480,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:30
17481,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00


In [21]:
df2.dtypes

id                        int64
2 Year_RTM  L            object
3 Day_RTM  L              int64
4 Hour_Minute_RTM  L     object
5 T108v_1_AVG  L        float64
6 T108v_2_AVG  L        float64
7 T108v_3_AVG  L        float64
8 T108v_4_AVG  L        float64
9 T108v_5_AVG  L        float64
10 T108v_6_AVG  L       float64
11 T108v_7_AVG  L       float64
12 T108v_8_AVG  L       float64
Tsuelo_10cm             float64
Tsuelo_20cm             float64
Tsuelo_30cm             float64
Tsuelo_40cm             float64
Tsuelo_55cm             float64
Tsuelo_70cm             float64
Tsuelo_85cm             float64
Tsuelo_100cm            float64
21 Tref_AVG  L          float64
22 period_1  L          float64
23 period_2  L          float64
24 period_3  L          float64
25 period_4  L          float64
26 period_5  L          float64
27 period_6  L          float64
28 period_7  L          float64
29 period_8  L          float64
30 TVWC_1  L            float64
                         ...   
36 TVWC_

In [22]:
for i in range (0,len(df2)):
    if(df2['4 Hour_Minute_RTM  L'][i]=='2400'):
        df2.replace(to_replace=df2['3 Day_RTM  L'][i],value=df2['3 Day_RTM  L'][i]+1)

In [23]:
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,THETA_30cm.1,THETA_40cm.1,THETA_55cm.1,THETA_70cm.1,THETA_85cm.1,THETA_100cm.1,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO
17477,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00
17478,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:30
17479,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:00
17480,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:30
17481,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00


In [24]:

d = []
for i in range (0,len(df2)):
    if((df2['TIEMPO'][i])=='00:00'):
        d = df2['3 Day_RTM  L'][i]+1
    else:
        d = df2['3 Day_RTM  L'][i]

In [25]:
#df2['DIA'] = d
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,THETA_30cm.1,THETA_40cm.1,THETA_55cm.1,THETA_70cm.1,THETA_85cm.1,THETA_100cm.1,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO
17477,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00
17478,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:30
17479,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:00
17480,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:30
17481,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00


In [26]:
#Creamos una variable del tipo string para la fecha que incluya el año, el número de día, la hora y el minuto
f = []   #Arreglo de fechas
for i in range (0,len(df2)):
    f.append('2009 '+str(df2['3 Day_RTM  L'][i])+ ' ' + h[i]+' '+m[i])

In [27]:

#Convertimos nuestro arreglo de fechas en una variable del tipo datetime
FECHA = []
for i in range(0,len(df2)):
    d=datetime.datetime.strptime(f[i],'%Y %j %H %M')
    F = d.isoformat(' ')
    FECHA.append(F)

In [28]:
#Agregamos la columna FECHA al dataframe
df2['F']=FECHA

In [29]:
df2['FECHA'] = pd.to_datetime(df2.apply(lambda x: x['F'], 1), dayfirst=True)
df2 = df2.drop(['F'], 1)
df2.head()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,THETA_40cm.1,THETA_55cm.1,THETA_70cm.1,THETA_85cm.1,THETA_100cm.1,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO,FECHA
0,147,2009,1,30,0.959,0.972,0.992,1.004,1.027,1.063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:30,2009-01-01 00:30:00
1,147,2009,1,100,0.958,0.972,0.993,1.004,1.027,1.063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:00,2009-01-01 01:00:00
2,147,2009,1,130,0.956,0.972,0.993,1.004,1.027,1.063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:30,2009-01-01 01:30:00
3,147,2009,1,200,0.955,0.972,0.993,1.005,1.027,1.063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2:00,2009-01-01 02:00:00
4,147,2009,1,230,0.953,0.972,0.994,1.005,1.027,1.063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2:30,2009-01-01 02:30:00


In [30]:
#Quitamos ahora las fechas duplicadas de nuestro segundo DataFrame
df2 = pd.DataFrame(df2.drop_duplicates(['FECHA']))
df2 = df2.reset_index(drop=True)
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,THETA_40cm.1,THETA_55cm.1,THETA_70cm.1,THETA_85cm.1,THETA_100cm.1,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO,FECHA
17421,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00,2009-12-31 22:00:00
17422,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:30,2009-12-31 22:30:00
17423,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:00,2009-12-31 23:00:00
17424,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:30,2009-12-31 23:30:00
17425,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00,2009-12-31 00:00:00


In [31]:
#Quitamos ahora las fechas duplicadas de nuestro primer DataFrame
df1 = pd.DataFrame(df1.drop_duplicates(['FECHA']))
df1 = df1.reset_index(drop=True)
df1.tail()

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA,MIN
17387,1.676,-0.139,14.36,1113,-79.21,NAN,NAN,3.067,14.7,0.407,1.672,0.681,5.126,NaN,2009-12-31 22:00:00,0.0
17388,1.196,-0.12,14.17,1170,-78.89,NAN,NAN,2.683,14.63,0.388,1.664,0.645,4.855,NaN,2009-12-31 22:30:00,30.0
17389,0.454,-0.074,12.63,1060,-72.47,NAN,NAN,2.419,13,0.472,1.497,0.707,5.354,NaN,2009-12-31 23:00:00,0.0
17390,0.746,-0.116,11.9,1043,-74.88,NAN,NAN,2.38,12.3,0.5,1.430,0.715,5.428,NaN,2009-12-31 23:30:00,30.0
17391,0.72,-0.078,11.02,1071,-70.44,NAN,NAN,2.068,11.28,0.521,1.336,0.696,5.299,NaN,2010-01-01 00:00:00,0.0


In [32]:

#Buscamos ahora las similitudes de FECHA entre nuestro primer y segundo DataFrame.
#Filtramos los DataFrames por las columnas que vamos a utilizar
#df1 = df1.filter(['FECHA','t_Avg','airT_Avg','net_rad_Avg'],axis=1)
#df2 = df2.filter(['FECHA','Tsuelo_10cm','Tsuelo_20cm','Tsuelo_30cm','Tsuelo_40cm','Tsuelo_55cm','Tsuelo_70cm','Tsuelo_85cm','Tsuelo_100cm'],axis=1)

In [33]:
#Unimos los DataFrames a partir de los valores coincidentes de la variable FECHA
#df = pd.merge(df1, df2, on=['FECHA'])

In [34]:
#df.tail()

In [35]:

df2.dtypes

id                               int64
2 Year_RTM  L                   object
3 Day_RTM  L                     int64
4 Hour_Minute_RTM  L            object
5 T108v_1_AVG  L               float64
6 T108v_2_AVG  L               float64
7 T108v_3_AVG  L               float64
8 T108v_4_AVG  L               float64
9 T108v_5_AVG  L               float64
10 T108v_6_AVG  L              float64
11 T108v_7_AVG  L              float64
12 T108v_8_AVG  L              float64
Tsuelo_10cm                    float64
Tsuelo_20cm                    float64
Tsuelo_30cm                    float64
Tsuelo_40cm                    float64
Tsuelo_55cm                    float64
Tsuelo_70cm                    float64
Tsuelo_85cm                    float64
Tsuelo_100cm                   float64
21 Tref_AVG  L                 float64
22 period_1  L                 float64
23 period_2  L                 float64
24 period_3  L                 float64
25 period_4  L                 float64
26 period_5  L           

In [36]:
df1.dtypes

u_Avg                  object
v_Avg                  object
w_Avg                  object
t_Avg                  object
kh20_Avg               object
net_rad_Avg            object
shf1_Avg               object
shf2_Avg               object
vv_Avg                 object
airT_Avg               object
rh_Avg                float64
e_sat_Avg              object
e_Avg                  object
h2o_hmp_Avg            object
FECHA          datetime64[ns]
MIN                   float64
dtype: object